In [1]:
%pip install opencv-python
%pip install pytesseract
!apt-get install tesseract-ocr
print("Lib's instaladas")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 0s (9,634 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 123597 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [5]:
import cv2
import pytesseract
from google.colab.patches import cv2_imshow
import os
import cv2

def encontrarRoiPlaca(source, arquivo, min_perimetro=250, threshold_value=90, blur_kernel=(5, 5), approx_poly_dp_factor=0.03, canny_low=50, canny_high=150, aspect_ratio_range=(2, 5), min_area=500):
    # Carrega a imagem
    img = cv2.imread(source)

    # Converte para escala de cinza
    cinza = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Aplica o limiar (binarização)
    _, bin = cv2.threshold(cinza, threshold_value, 255, cv2.THRESH_BINARY)

    # Aplica desfoque gaussiano
    desfoque = cv2.GaussianBlur(bin, blur_kernel, 0)

    # Aplica o detector de bordas de Canny
    edges = cv2.Canny(desfoque, canny_low, canny_high)

    # Encontra contornos na imagem binarizada
    contornos, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    placas_detectadas = []

    # Criar pasta com o nome do arquivo
    pasta = f'/content/drive/MyDrive/ColabNotebooks/tcc_mba/detectando_placa_opecv/{arquivo}'
    os.makedirs(pasta, exist_ok=True)

    # Salvar a imagem original na pasta
    cv2.imwrite(f'{pasta}/{arquivo}.jpg', img)

    for c in contornos:
        perimetro = cv2.arcLength(c, True)
        if perimetro > min_perimetro:
            aprox = cv2.approxPolyDP(c, approx_poly_dp_factor * perimetro, True)
            if len(aprox) == 4:  # Verifica se o contorno é um quadrilátero
                (x, y, lar, alt) = cv2.boundingRect(c)
                aspect_ratio = lar / float(alt)
                area = cv2.contourArea(c)

                # Filtra por proporção e área
                if aspect_ratio_range[0] <= aspect_ratio <= aspect_ratio_range[1] and area > min_area:
                    cv2.rectangle(img, (x, y), (x + lar, y + alt), (0, 255, 0), 2)
                    roi = img[y:y + alt, x:x + lar]
                    placas_detectadas.append(roi)
                    cv2.imwrite(f'{pasta}/{arquivo}_detec.png', roi)

    # Exibe as placas detectadas
    if placas_detectadas:
        for placa in placas_detectadas:
            print("Placa detectada:")
    else:
        print("Nenhuma placa detectada.")

    # Exibe a imagem final com as placas destacadas
    cv2_imshow(img)

def processar_imagens_na_pasta(pasta):
    for arquivo in os.listdir(pasta):
        if arquivo.endswith(".jpg") or arquivo.endswith(".png"):
            print(f"Processando: {arquivo}")
            caminho_completo = os.path.join(pasta, arquivo)
            encontrarRoiPlaca(caminho_completo, arquivo)


if __name__ == "__main__":
    processar_imagens_na_pasta("/content/drive/MyDrive/ColabNotebooks/tcc_mba/placas_exemplos_para_deteccao")

Output hidden; open in https://colab.research.google.com to view.

35 placas detectadas com algumas tem que realizar tratamento
tempo de execução 1:15 minutos